In [1]:
from utils import AiDevsTask, OpenAIChat
from utils.tools import NBPTools, RestCountriesTools
import json
import os
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

# Usage example:
task_name = "knowledge"
task = AiDevsTask()
task.get_token(task_name)
task_response = task.get_task_content()

task_msg = task_response['msg']
print("Task message: ", task_msg)
task_question = task_response['question']
print("Pytanie: ", task_question)
task_database1 = task_response['database #1']
print("Baza 1: ", task_database1)
task_database2 = task_response['database #2']
print("Baza 2: ", task_database2)

json_example = """
"What is the dollar exchange rate in relation to the Polish zloty?" -> {"tool_name": "nbp", "parameter": "usd"}
"What is the euro exchange rate?" -> {"tool_name": "nbp", "parameter": "eur"}
"What is the population of italy?" -> {"tool_name": "rest_countries", "parameter": "italy"}
"what is the population of poland?" -> {"tool_name": "rest_countries", "parameter": "poland"}
"what is the capital of france?" -> {"tool_name": "chat", "parameter": ""}
"explain the theory of relativity" -> {"tool_name": "chat", "parameter": ""}
"""

chatbot = OpenAIChat()
messages=[
        HumanMessage(content=f"""{task_question}. Return the tool that should be used to answer the question and value that needs to be used in the and nothing else. Use JSON format.
                     ###Examples:
                     {json_example}
                     """),
    ]
chat_response = chatbot.handle_messages(messages)
chat_response = json.loads(chat_response)
print("Chat response: ", chat_response)

tool_name = chat_response["tool_name"]
parameter = chat_response["parameter"]

context = ""
match tool_name:
    case "nbp":
        exchange_rate = NBPTools.get_exchange_rates(table="A", code=parameter)
        context = f"""###Context: Exchange Rate of {parameter} is {exchange_rate}.###"""
    case "rest_countries":
        population = RestCountriesTools.get_country_info(name=parameter)
        context = f"""###Context: Population of {parameter} is {population}.###"""
    case "chat":
        context = "###Context: Answer based on your knowldedge.###"

chatbot = OpenAIChat()
messages=[
        HumanMessage(content=f"""{task_question}. Return answer to the question and nothing else. Answer in Polish Language. Refer to context if needed.
            {context}"""),
    ]
chat_response = chatbot.handle_messages(messages)
print("Chat response: ", chat_response)

task.send_answer(chat_response)

Token: 9b0354ef4e1a27ff82d018e34536905ed71272e3
Treść zadania:  {'code': 0, 'msg': 'I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model', 'question': 'podaj aktualny kurs EURO', 'database #1': 'Currency http://api.nbp.pl/en.html (use table A)', 'database #2': "Knowledge about countries https://restcountries.com/ - field 'population'"}
Task message:  I will ask you a question about the exchange rate, the current population or general knowledge. Decide whether you will take your knowledge from external sources or from the knowledge of the model
Pytanie:  podaj aktualny kurs EURO
Baza 1:  Currency http://api.nbp.pl/en.html (use table A)
Baza 2:  Knowledge about countries https://restcountries.com/ - field 'population'


c:\Users\dkwar\Desktop\Forge\samurai-api\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Chat response:  {'tool_name': 'nbp', 'parameter': 'eur'}
http://api.nbp.pl/api/exchangerates/rates/A/eur
Chat response:  Kurs EURO wynosi 4.3316.
Odpowiedź: {'answer': 'Kurs EURO wynosi 4.3316.'}
Odpowiedź serwera:  {'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}